# Uso de uma RNA (CNN) para o diagnóstico de glaucoma #
### Alunos: ###
- Miguel Mendes Luna - 211026501
- Pedro Eduardo - 
- Davy - 

In [ ]:
# Importações básicas
import numpy as np
import pandas as pd
import os

# Explorando uma imagem

Fazendo algumas verificações acerca de um exemplo de imagem, tais como seu formato, seu tamanho (resolução), a imagem propriamente dita e uma representação matricial dos seus pixels com base no código RGB.

In [ ]:
from numpy import asarray

from PIL import Image

# Carregando um exemplo de imagem
image = Image.open('/kaggle/input/glaucoma-detection/Fundus_Train_Val_Data/Fundus_Scanes_Sorted/Validation/Glaucoma_Positive/613.jpg') 

# Imprimindo alguns dados acerca da imagem
print("Formato da imagem: " + str(image.format))
print("Modo da imagem: " + str(image.mode))
print("Tamanho da imagem (pixels): " + str(image.size))

# Mostrando a imagem
display(image)

# Mostrando os pixels da imagem matricialmente
pixels = asarray(image)
print(pixels)

# Tratamento de pixels

Aqui é feita a normalização do código RGB dos pixels para que representem um número entre -1 e 1. Note que, inicialmente, é calculada a média geral dos valores correspondentes dos pixels, para então subtraí-la de cada valor, a fim de deixar os valores resultantes "ao redor" de zero, de forma que a média global passou a ser nula. Em seguida, dividiu-se todos os valores correspondentes por 255 (máximo valor no RGB), para que os valores ficassem entre 0 e 1 em módulo.

In [ ]:
# global centering

# calculate global mean
mean = pixels.mean()
print('Mean: %.3f' % mean)
print('Min: %.3f, Max: %.3f' % (pixels.min(), pixels.max()))
# global centering of pixels
pixels = pixels - mean
# confirm it had the desired effect
mean = pixels.mean()
print('Mean: %.3f' % mean)
print('Min: %.3f, Max: %.3f' % (pixels.min(), pixels.max()))
print(pixels)


# example of pixel normalization
# confirm pixel range is 0-255
print('Data Type: %s' % pixels.dtype)
print('Min: %.3f, Max: %.3f' % (pixels.min(), pixels.max()))
# convert from integers to floats
pixels = pixels.astype('float32')
# normalize to the range 0-1
pixels /= 255.0
mean = pixels.mean()
print('pixel mean = ', mean)

# confirm the normalization
print('Min: %.3f, Max: %.3f' % (pixels.min(), pixels.max()))
print(pixels)

# Visualização parcial

Visualização da imagem original quando comparada à imagem após a normalização dos pixels.

In [ ]:
# Visualization
import matplotlib.pyplot as plt
fig, (ax0, ax1) = plt.subplots(1, 2)
ax0.imshow(image)
ax0.axis('off')
ax0.set_title('image')
ax1.imshow(pixels)
ax1.axis('off')
ax1.set_title('result')
plt.show()

In [ ]:
# Declaração de diretórios
TRAIN_DIR = '/kaggle/input/glaucoma-detection/Fundus_Train_Val_Data/Fundus_Scanes_Sorted/Train'
TEST_DIR = '/kaggle/input/glaucoma-detection/Fundus_Train_Val_Data/Fundus_Scanes_Sorted/Validation'

In [ ]:
# Construindo a rede neural

from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential, Model 
from keras.optimizers import SGD, Adam
from keras.callbacks import TensorBoard
from keras import metrics
import keras
import matplotlib.pyplot as plt

HEIGHT = 300
WIDTH = 300

BATCH_SIZE = 8
class_list = ["class_1", "class_2"]
FC_LAYERS = [1024, 512, 256]
dropout = 0.5
NUM_EPOCHS = 100
BATCH_SIZE = 8

def build_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    for fc in fc_layers:
        # print(fc)
        x = Dense(fc, activation='relu')(x)
        x = Dropout(dropout)(x)
    preditions = Dense(num_classes, activation='softmax')(x)
    finetune_model = Model(inputs = base_model.input, outputs = preditions)
    return finetune_model

base_model_1 = ResNet50(weights = 'imagenet',
                       include_top = False,
                       input_shape = (HEIGHT, WIDTH, 3))

train_datagen = ImageDataGenerator(preprocessing_function = preprocess_input,
                                   rotation_range = 90,
                                   horizontal_flip = True,
                                   vertical_flip = True,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.1,)

test_datagen = ImageDataGenerator(preprocessing_function = preprocess_input,
                                  rotation_range = 90,
                                  horizontal_flip = True,
                                  vertical_flip = False)

train_generator = train_datagen.flow_from_directory(TRAIN_DIR,
                                                    target_size = (HEIGHT, WIDTH),
                                                    batch_size = BATCH_SIZE)

test_generator = test_datagen.flow_from_directory(TEST_DIR,
                                                  target_size = (HEIGHT, WIDTH),
                                                  batch_size = BATCH_SIZE)




resnet50_model = build_model(base_model_1,
                                      dropout = dropout,
                                      fc_layers = FC_LAYERS,
                                      num_classes = len(class_list))

adam = Adam(learning_rate = 0.00001)
resnet50_model.compile(adam, loss="binary_crossentropy", metrics=["accuracy", metrics.mean_squared_error, metrics.F1Score])

filepath = "./checkpoints" + "RestNet50" + "_model_weights.keras"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor = ["acc"], verbose= 1, mode = "max")
cb=TensorBoard(log_dir=("/home/ubuntu/"))
callbacks_list = [checkpoint, cb]

# print(train_generator.class_indices)

# resnet50_model.summary()

In [ ]:
# Treinamento
history = resnet50_model.fit(train_generator, epochs = NUM_EPOCHS, steps_per_epoch = 100, 
                                       shuffle = True, validation_data = test_generator)

In [ ]:
import matplotlib as pyplot
pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['f1_score'])
pyplot.plot(history.history['mean_squared_error'])
pyplot.show()